<h1>ETL Project Notebook # 1</h1>
<h3>Authors: John Jostes and Tim Lucas</h3>
<p>In this notebook we are reading in the TSV files we downloaded from the IMDB website. Once we loaded this data into a pandas data frame we were able to remove unnecessary columns, filter the data for years within the 1970s, title types of "Movies" and genres containing "Action." Once this data was narrowed down we did a merge with the ratings file and wrote to a CSV so that we could read into a second notebook to handle additional extraction via OMDB API.</p>

In [1]:
import pandas as pd

In [ ]:
#creating first db
filepath = 'Resources/title_basics.tsv'

file = pd.read_csv(filepath, sep='\t')

basics_df = pd.DataFrame(file)

In [3]:
#creating second db
filepath = 'Resources/title_ratings.tsv'

file = pd.read_csv(filepath, sep='\t')

ratings_df = pd.DataFrame(file)

In [4]:
#limiting data to non-adult & movies only
movies_df = basics_df.loc[(basics_df['titleType'] == 'movie') & (basics_df['isAdult'] == 0), :]

movies_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
332,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
499,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
571,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama"


In [5]:
#removing columns to simplify db

movies_df = movies_df.drop(['isAdult', 'titleType', 'endYear', 'originalTitle'], axis=1)
movies_df.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres
8,tt0000009,Miss Jerry,1894,45,Romance
145,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport"
332,tt0000335,Soldiers of the Cross,1900,\N,"Biography,Drama"
499,tt0000502,Bohemios,1905,100,\N
571,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama"


In [6]:
#removing Null startYear values

movies_df = movies_df.loc[movies_df['startYear'] != '\\N', :]

#filtering for movies from the 1970's
movies_df['startYear'] = pd.to_numeric(movies_df['startYear'])
movies_df = movies_df.loc[(movies_df['startYear'] >= 1970) & (movies_df['startYear'] < 1980), :]

In [16]:
#filtering for genres that include, but not limited to, Action
action_movies = movies_df[movies_df['genres'].str.contains('Action')]

#dropping null values
action_movies = action_movies.loc[(action_movies['runtimeMinutes'] != '\\N') & (action_movies['genres'] != '\\N'), :]

In [17]:
#reset index
action_movies = action_movies.sort_values('tconst')
action_movies = action_movies.reset_index(drop='True')

In [18]:
#merging db's
action_movies = pd.merge(action_movies, ratings_df, on='tconst')

In [19]:
action_movies

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0058994,Bruce's Fingers,1976,91,"Action,Crime,Drama",5.2,132
1,tt0059680,Satan's Harvest,1970,88,Action,3.9,102
2,tt0060462,The Girls from Thunder Strip,1970,80,"Action,Drama",4.4,89
3,tt0060939,Santo vs. the Head Hunters,1971,80,"Action,Adventure,Drama",5.1,31
4,tt0060951,A Scream in the Streets,1973,96,"Action,Crime",4.7,226
...,...,...,...,...,...,...,...
2335,tt7063254,Ryûketsu no kôsô,1971,86,"Action,Crime",6.6,8
2336,tt7383548,The Magnificent Seven,1973,78,"Action,Drama,Thriller",5.4,8
2337,tt7464002,Bandari,1973,91,"Action,Musical,Romance",5.8,5
2338,tt7705702,Two Graves to Kung Fu,1974,91,Action,4.7,7


In [20]:
action_movies.to_csv('Output/action_movies.csv')